### Restart and Run All Cells

In [1]:
import glob
import os
import pandas as pd
from datetime import date, timedelta
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

format_dict = {'qty':'{:,}','volbuy':'{:,}',
              'price_x':'{:,.2f}','price_y':'{:,.2f}','change':'{:,.2f}','price':'{:,.2f}',
              'chg_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','amount':'{:,.2f}',
              'percent':'{:,.2f}','net':'{:,.2f}','dividend':'{:,.4f}'}

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"
dts_path = "/Users/User/Downloads/Datasets/"

pd.options.display.max_rows = 11
pd.options.display.float_format = '{:.2f}'.format
year = 2023
month = 8
today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 10, 12), datetime.date(2023, 10, 11))

In [2]:
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2023-10-12
yesterday: 2023-10-11


## Must rerun whenever there is change in orders

In [3]:
sql = """
SELECT name, date, volbuy, price, active, dividend, period, grade 
FROM buy
ORDER BY name"""
stocks = pd.read_sql(sql, const)
stocks['volbuy'] = stocks['volbuy'].astype('int64')
stocks.style.format(format_dict)

,name,date,volbuy,price,active,dividend,period,grade
0,AH,2023-06-08,"1,200",37.00,1,1.6600,3,A3
1,AIMIRT,2023-08-17,"10,000",11.00,1,0.8920,2,A3
2,ASK,2023-02-16,"4,500",31.00,1,1.4400,3,A4
3,ASP,2022-03-07,"30,000",3.80,1,0.2100,2,B4
4,AWC,2023-06-15,"9,000",4.96,1,nan,3,A1
5,BCH,2021-09-07,"15,000",21.46,1,0.5000,4,C2
6,CPNREIT,2022-08-16,"60,000",17.55,1,0.8977,2,A2
7,DIF,2020-08-01,"30,000",14.70,1,0.9955,2,C3
8,GVREIT,2022-08-24,"60,000",7.90,1,0.7925,2,B3
9,IVL,2022-08-29,"7,200",40.00,1,1.3000,4,C3


In [4]:
names = stocks["name"].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'AH', 'AIMIRT', 'ASK', 'ASP', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'TOA', 'WHAIR', 'WHART'"

In [5]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
input_file

'../data/Price-Today.csv'

In [6]:
prices = pd.read_csv(input_file)
prices.shape

(190, 10)

In [7]:
df_merge  = pd.merge(stocks, prices, on = 'name', how='inner')
df_merge.shape

(30, 17)

### Convert column name to make it valid for df.query

In [8]:
pd.options.display.max_rows = 30
df_merge['percent'] = df_merge['%change']
df_merge['chg_amt'] = df_merge['volbuy'] * df_merge['change']
df_merge['volbuy'] = df_merge['volbuy'].astype('int64')
colo = ['name', 'volbuy', 'price_x', 'price_y', 'change', 'percent', 'chg_amt']
filter = df_merge['percent'] != 0
df_pct = df_merge[filter][colo].sort_values(by='percent', ascending=True).copy()
df_amt = df_merge[filter][colo].sort_values(by='chg_amt', ascending=True).copy()

In [9]:
df_pct.style.format(format_dict)

,name,volbuy,price_x,price_y,change,percent,chg_amt
12,JMT,"4,200",55.50,38.00,-2.75,-6.75,"-11,550.00"
22,SINGER,"6,000",24.80,11.00,-0.60,-5.17,"-3,600.00"
11,JMART,"6,800",33.00,21.00,-0.90,-4.11,"-6,120.00"
24,SYNEX,"15,000",28.70,11.20,-0.20,-1.75,"-3,000.00"
18,PTTGC,"6,000",64.75,33.75,-0.50,-1.46,"-3,000.00"
19,RCL,"27,000",38.75,21.30,-0.30,-1.39,"-8,100.00"
13,KCE,"10,000",77.25,55.50,-0.75,-1.33,"-7,500.00"
17,PTG,"3,600",11.40,8.90,-0.10,-1.11,-360.00
29,WHART,"20,000",12.30,9.60,-0.10,-1.03,"-2,000.00"
1,AIMIRT,"10,000",11.00,10.40,-0.10,-0.95,"-1,000.00"


In [10]:
df_amt.style.format(format_dict)

,name,volbuy,price_x,price_y,change,percent,chg_amt
12,JMT,"4,200",55.50,38.00,-2.75,-6.75,"-11,550.00"
19,RCL,"27,000",38.75,21.30,-0.30,-1.39,"-8,100.00"
13,KCE,"10,000",77.25,55.50,-0.75,-1.33,"-7,500.00"
11,JMART,"6,800",33.00,21.00,-0.90,-4.11,"-6,120.00"
14,MCS,"78,000",15.10,6.65,-0.05,-0.75,"-3,900.00"
22,SINGER,"6,000",24.80,11.00,-0.60,-5.17,"-3,600.00"
18,PTTGC,"6,000",64.75,33.75,-0.50,-1.46,"-3,000.00"
24,SYNEX,"15,000",28.70,11.20,-0.20,-1.75,"-3,000.00"
6,CPNREIT,"60,000",17.55,9.95,-0.05,-0.50,"-3,000.00"
8,GVREIT,"60,000",7.90,5.75,-0.05,-0.86,"-3,000.00"


In [11]:
df_pct.query('percent >= 3.00').style.format(format_dict)

,name,volbuy,price_x,price_y,change,percent,chg_amt


In [12]:
df_pct.query('percent < -3.00').style.format(format_dict)

,name,volbuy,price_x,price_y,change,percent,chg_amt
12,JMT,"4,200",55.50,38.00,-2.75,-6.75,"-11,550.00"
22,SINGER,"6,000",24.80,11.00,-0.60,-5.17,"-3,600.00"
11,JMART,"6,800",33.00,21.00,-0.90,-4.11,"-6,120.00"


In [13]:
df_amt.query('chg_amt >= 6000').style.format(format_dict)

,name,volbuy,price_x,price_y,change,percent,chg_amt


In [14]:
df_amt.query('chg_amt < -6000').style.format(format_dict)

,name,volbuy,price_x,price_y,change,percent,chg_amt
12,JMT,"4,200",55.50,38.00,-2.75,-6.75,"-11,550.00"
19,RCL,"27,000",38.75,21.30,-0.30,-1.39,"-8,100.00"
13,KCE,"10,000",77.25,55.50,-0.75,-1.33,"-7,500.00"
11,JMART,"6,800",33.00,21.00,-0.90,-4.11,"-6,120.00"


In [15]:
float_formatter = "฿{:,.2f}"
float_formatter.format(df_amt.chg_amt.sum())

'฿-56,360.00'